# Recognition gap: Search for MIRCs

This notebook contains the code for the main experiment of the third
case study "recognition gap" in "The Notorious Difficult of Comparing
Human and Machine Perception" (Funke, Borowski et al. 2020): We
implement a search algorithm for a deep convolutional neural network to
identify MIRCs (minimal recognizable configuration). The procedure is
very similar to the human experiment performed by Ullman et al. (2016).

## Libraries, packages, ...

In [ ]:
# basic imports
import os

# standard libraries
import numpy as np
import math
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import csv
import time

# torch imports
import torch

# custom imports
import configuration_for_experiment as config
import utils.pytorchnet_bagnets as pytorchnet_bagnets
import utils.data_in as data_in
import utils.data_out as data_out
import utils.search as search

## Set device

In [ ]:
# set device on GPU if available, else CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load data

In [ ]:
# get data_loader
data_loader = data_in.get_data_loader(config.Ullman_or_ImageNet)

## Load model

In [ ]:
model = pytorchnet_bagnets.bagnet33(pretrained=True).to(DEVICE)
model.avg_pool = False
model.eval()
torch.set_grad_enabled(False)

## Directories for ouput data

In [ ]:
exp_dir = data_out.make_exp_dir(
    config.Ullman_or_ImageNet,
    config.list_as_one_class,
    config.start_idx,
    config.stop_idx,
    config.descendent_specifier)

## Do it! Search MIRCs - and while you're at it, also sub-MIRCs

In [ ]:
write_or_append = "w"
start = time.time()
# loop through all images in data_loader
# note that this data_loader was slightly modified and that it returns a
# list of target(s) and the path to the image file
for number_IN, (image_from_loader, target, path) in enumerate(data_loader):
    # only perform the search if the images are from Ullman et al.
    # or if the images are in the specified range (start_idx and stop_idx)
    if (
        # and ("suit" in path[0])) # TODO
        ((config.Ullman_or_ImageNet == "Ullman"))
        or ((config.Ullman_or_ImageNet == "ImageNet") and (number_IN >= config.start_idx) and (number_IN < config.stop_idx))
    ):
        print("\nnumber_IN", number_IN)

        # all classes as one class
        if config.list_as_one_class:
            target_list = target
            search.perform_MIRC_search(
                image_from_loader,
                target_list,
                path,
                model,
                DEVICE,
                config.Ullman_or_ImageNet,
                config.descendent_specifier,
                exp_dir,
                write_or_append)
            write_or_append = "a"
        # individual elements as separate classes
        else:
            for target_i in target:
                target_list = [target_i]
                search.perform_MIRC_search(
                    image_from_loader,
                    target_list,
                    path,
                    model,
                    DEVICE,
                    config.Ullman_or_ImageNet,
                    config.descendent_specifier,
                    exp_dir,
                    write_or_append)
                write_or_append = "a"

print("done")
stop = time.time()
print(f"time {stop-start}")